# dataset check 

> Before or after training, we want to check dataset, this checking functionality will be used here

In [3]:
#| default_exp dataset_check

In [4]:
#| export
import os
import shutil
from IPython.display import display, clear_output
import io
from PIL import Image as Image
from ipywidgets import (widgets, Button,
                        HBox, VBox, Layout)
from ipywidgets import Image as WImage
from pathlib import Path
import numpy as np
from fastcore.all import *
from typing import List, Union, Any, Callable

In [3]:
#| export
def display_image_row(
        im_path:Union[Path, str],
        move_path:Union[Path, str],
        max_images:int=10,
        start:int=0,
        im_height:int=100,
        im_width:int=100,
    ): 

    def on_next_click(b):
        nonlocal start

        start = start + max_images
        print(f' start = {start}')
        clear_output(wait=True)
        display_image_row(
            im_path,
            move_path,
            max_images,
            start, 
            im_height,
            im_width
        )
    def on_prev_click(b):
        nonlocal start
        start = max( 0, start - max_images)
        clear_output(wait=True)
        display_image_row(
            im_path,
            move_path,
            max_images,
            start,
            im_height,
            im_width
        )
    def on_move_click(
            b,
            file_name):
        source_path = Path(im_path, file_name)
        destination_path = Path(move_path, file_name)
        shutil.move(source_path, destination_path)
        clear_output(wait=True)
        display_image_row(
            im_path,
            move_path,
            max_images,
            start,
            im_height,
            im_width
        )
    clear_output(wait=True)

    files = sorted(im_path.ls(file_exts=['.png', '.tif']))
    files_to_d = files[start:start + max_images]

    image_boxes = []
    
    for file in files_to_d:
        name_ = Path(file).name
        image_path = Path(im_path, name_)

        with Image.open(image_path) as pil_img:
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='PNG')
            img_widget = WImage(
                value=img_byte_arr.getvalue(),
                format='png',
                width=im_width,
                height=im_height
                )
            
            btn_del = Button(description='Delete')
            btn_move = Button(description='Move')

            btn_move.on_click(
                lambda b, f=name_: on_move_click(b, f)
            )

            box = VBox(
                [
                    img_widget,
                    HBox([btn_del, btn_move])
                    ])
            image_boxes.append(box)


    btn_next = Button(description='Next') 
    btn_prev = Button(description='Prev') 

    if start > 0 : 
        print(f'start = {start}')
        print(f'max = {max_images}')
        btn_prev.on_click(on_prev_click)
    if start + max_images < len(files):
        btn_next.on_click(on_next_click)

    nav = HBox([btn_prev, btn_next])
    display(
        HBox(image_boxes, Layout=Layout(
            flex_flow='row_wrap', 
            align_items='center',
        ))
    )
    display(nav)


        


In [6]:
HOME = Path.home()
crop_im_path = Path(fr'{HOME}/Schreibtisch/projects/data/easy_front/Cropped_Images_png')
mask_path = Path(fr'{HOME}/Schreibtisch/projects/data/easy_front/per_sam_masks')
overlay_mask_path = Path(fr'{HOME}/Schreibtisch/projects/data/easy_front/overlay_masks_old_data')
overlay_mask_path.mkdir(exist_ok=True, parents=True)
crop_im_path.ls(), mask_path.ls()

((#1639) [Path('/home/user/Schreibtisch/projects/data/easy_front/Cropped_Images_png/2_1_CroppedImg_20231018_92536201_5_A.png'),Path('/home/user/Schreibtisch/projects/data/easy_front/Cropped_Images_png/1_4_CroppedImg_20231018_92329777_33_B.png'),Path('/home/user/Schreibtisch/projects/data/easy_front/Cropped_Images_png/2_1_CroppedImg_20231018_92536201_25_A.png'),Path('/home/user/Schreibtisch/projects/data/easy_front/Cropped_Images_png/2_2_CroppedImg_20231018_92536201_24_A.png'),Path('/home/user/Schreibtisch/projects/data/easy_front/Cropped_Images_png/3_4_CroppedImg_20231018_92716715_21_B.png'),Path('/home/user/Schreibtisch/projects/data/easy_front/Cropped_Images_png/0_1_CroppedImg_20231018_92042317_7_A.png'),Path('/home/user/Schreibtisch/projects/data/easy_front/Cropped_Images_png/3_3_CroppedImg_20231018_92716715_26_B.png'),Path('/home/user/Schreibtisch/projects/data/easy_front/Cropped_Images_png/2_2_CroppedImg_20231018_92536201_17_A.png'),Path('/home/user/Schreibtisch/projects/data/easy

In [7]:
from cv_tools.core import *
mask_names = get_name_(mask_path.ls())
image_names = get_name_(crop_im_path.ls())

In [8]:
common_names = set(image_names).intersection(set(mask_names))

In [9]:
for i in common_names:
    sample_im = Path(crop_im_path,i)
    sample_msk = Path(mask_path,i)
    overlay_mask_border_on_image(
        im_path=sample_im,
        msk_path=sample_msk,
        save_overlay_img_path=overlay_mask_path,

    )

/home/user/Schreibtisch/projects/git_data/cv_tools/cv_tools/core.py:237: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(scale_*rgb_image.shape[1] / dpi, scale_*rgb_image.shape[0] / dpi))


In [6]:
#| hide
import nbdev; nbdev.nbdev_export()